In [ ]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U seqeval
!pip install -U evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 21.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0

In [ ]:
import pandas as pd
from datasets import load_dataset

In [ ]:
data = load_dataset('PranavaKailash/CyNER2.0_augmented_dataset')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/241k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/236k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7751 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1661 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1662 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 7751
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1661
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1662
    })
})

In [ ]:
data['train']['tokens'][0]

['We',
 'believe',
 'the',
 'TelePort',
 'Crew',
 'Threat',
 'Actor',
 'is',
 'operating',
 'out',
 'of',
 'Russia',
 'or',
 'Eastern',
 'Europe',
 'with',
 'the',
 'groups',
 'major',
 'motivations',
 'appearing',
 'to',
 'be',
 'financial',
 'in',
 'nature',
 'through',
 'cybercrime',
 'and/or',
 'corporate',
 'espionage.']

In [ ]:
data['train']

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 7751
})

In [ ]:
data['train']['ner_tags'][0]

[16,
 16,
 16,
 6,
 14,
 14,
 14,
 16,
 16,
 16,
 16,
 2,
 16,
 2,
 10,
 16,
 16,
 6,
 16,
 16,
 16,
 16,
 16,
 4,
 16,
 16,
 16,
 6,
 16,
 6,
 14]

In [ ]:
from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer
)
import numpy as np

In [ ]:
model_checkpoint = "PranavaKailash/CyNER-2.0-DeBERTa-v3-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
label_list = data["train"].features["ner_tags"].feature.names

print("Labels:", label_list)
num_labels = len(label_list)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Labels: ['B-Date', 'B-Indicator', 'B-Location', 'B-Malware', 'B-Organization', 'B-System', 'B-Threat_group', 'B-Vulnerability', 'I-Date', 'I-Indicator', 'I-Location', 'I-Malware', 'I-Organization', 'I-System', 'I-Threat_group', 'I-Vulnerability', 'O']


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, padding=False
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/7751 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1661 [00:00<?, ? examples/s]

Map:   0%|          | 0/1662 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label={i: l for i, l in enumerate(label_list)},
    label2id={l: i for i, l in enumerate(label_list)}
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/735M [00:00<?, ?B/s]

In [ ]:
import evaluate

metric = evaluate.load("seqeval")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./cyner_ner_model",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to="none",
    logging_steps=100
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-1146666687.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Step,Training Loss
100,0.155700
200,0.134200
300,0.105200
400,0.179900
500,0.126800
600,0.137500
700,0.180200
800,0.191900
900,0.177100
1000,0.209400


TrainOutput(global_step=7751, training_loss=0.14376533983476913, metrics={'train_runtime': 1599.9289, 'train_samples_per_second': 4.845, 'train_steps_per_second': 4.845, 'total_flos': 281571635763522.0, 'train_loss': 0.14376533983476913, 'epoch': 1.0})

In [ ]:
metrics = trainer.evaluate(tokenized_datasets["test"])
print(metrics)

{'eval_loss': 0.2064172327518463, 'eval_precision': 0.9165408860871881, 'eval_recall': 0.930754665945361, 'eval_f1': 0.9235930929587546, 'eval_accuracy': 0.9504852815699659, 'eval_runtime': 43.3806, 'eval_samples_per_second': 38.312, 'eval_steps_per_second': 38.312, 'epoch': 1.0}


In [ ]:
trainer.save_model("./final_cyner_model")

from transformers import pipeline

ner_pipeline = pipeline(
    "token-classification",
    model="./final_cyner_model",
    tokenizer=tokenizer,
    grouped_entities=True
)

text = "APT29 used a custom malware to exploit the Windows RPC vulnerability."
result = ner_pipeline(text)
print(result)

Device set to use cuda:0


[{'entity_group': 'Threat_group', 'score': np.float32(0.98560303), 'word': 'APT29', 'start': 0, 'end': 5}, {'entity_group': 'Malware', 'score': np.float32(0.98199093), 'word': 'custom malware', 'start': 12, 'end': 27}, {'entity_group': 'Vulnerability', 'score': np.float32(0.6007804), 'word': 'exploit the', 'start': 30, 'end': 42}, {'entity_group': 'Vulnerability', 'score': np.float32(0.8038419), 'word': 'Windows RPC vulnerability', 'start': 42, 'end': 68}, {'entity_group': 'Indicator', 'score': np.float32(0.110028625), 'word': '.', 'start': 68, 'end': 69}]


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [ ]:
text = "Emotet used a malicious Word document to download TrickBot."
result = ner_pipeline(text)
print(result)

[{'entity_group': 'Malware', 'score': np.float32(0.6187826), 'word': 'Emotet', 'start': 0, 'end': 6}, {'entity_group': 'Indicator', 'score': np.float32(0.9745615), 'word': 'malicious Word document', 'start': 13, 'end': 37}, {'entity_group': 'Malware', 'score': np.float32(0.9940731), 'word': 'TrickBot', 'start': 49, 'end': 58}]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import os

# Define the source and destination paths
source_dir = "./final_cyner_model"
destination_dir = "/content/drive/MyDrive/final_cyner_model"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Copy the contents of the source directory to the destination directory
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

print(f"Model saved to: {destination_dir}")

Model saved to: /content/drive/MyDrive/final_cyner_model


In [ ]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.push_to_hub("cyner-ner-model")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...cyner_ner_model/spm.model: 100%|##########| 2.46MB / 2.46MB            

  ...r_model/model.safetensors:   0%|          | 18.2kB /  735MB            

  ...r_model/training_args.bin:   1%|          |  49.0B / 5.78kB            

CommitInfo(commit_url='https://huggingface.co/UnknownNemesis/cyner_ner_model/commit/a52701a2c4944f8b3dd3670ce053f14547c401b0', commit_message='cyner-ner-model', commit_description='', oid='a52701a2c4944f8b3dd3670ce053f14547c401b0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/UnknownNemesis/cyner_ner_model', endpoint='https://huggingface.co', repo_type='model', repo_id='UnknownNemesis/cyner_ner_model'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

ner_pipeline = pipeline(
    "token-classification",
    model="UnknownNemesis/cyner_ner_model",
    tokenizer="UnknownNemesis/cyner_ner_model",
    grouped_entities=True
)

print(ner_pipeline("APT29 exploited CVE-2021-44228 in a recent attack."))

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/735M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'Threat_group', 'score': np.float32(0.98500615), 'word': 'APT29', 'start': 0, 'end': 5}, {'entity_group': 'Vulnerability', 'score': np.float32(0.8402378), 'word': 'exploited', 'start': 5, 'end': 15}, {'entity_group': 'Indicator', 'score': np.float32(0.8106702), 'word': 'CVE-2021-44228', 'start': 15, 'end': 30}, {'entity_group': 'Indicator', 'score': np.float32(0.7945287), 'word': 'attack', 'start': 42, 'end': 49}]
